### 
> - **To sell more credit card products to Bank customers**
> -
> - To identify the potential customers who have a higher probability of purchasing the card so that     we can recommend that particular card to user.

In [ ]:
import numpy as np # 
import pandas as pd # 


import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from datetime import datetime
from datetime import date

import warnings

warnings.filterwarnings("ignore")

In [ ]:
import os
print(os.listdir())

['.config', 'completedtrans.csv', 'completedloan.csv', 'completeddistrict.csv', 'completedclient.csv', 'CRM Reviews.xlsx', 'CRM Events.xlsx', 'completedorder.csv', 'CRM Reviews.csv', 'CRM Call Center Logs.csv', 'completeddisposition.csv', 'LuxuryLoanPortfolio.csv', 'completedacct.csv', 'CRM Events.csv', 'completedcard.csv', 'sample_data']


### **Loading of Dataframes**

In [ ]:
accounts_df = pd.read_csv ('completedacct.csv', sep = ',')
cards_df = pd.read_csv ('completedcard.csv', sep = ',')
clients_df = pd.read_csv ('completedclient.csv', sep = ',')
dispos_df = pd.read_csv ('completeddisposition.csv', sep = ',')
district_df = pd.read_csv ('completeddistrict.csv', sep = ',')
loan_df = pd.read_csv ('completedloan.csv', sep = ',')
order_df = pd.read_csv ('completedorder.csv', sep = ',')
trans_df = pd.read_csv ('completedtrans.csv', sep = ',')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
trans_df ['operation'].fillna ('INTEREST_CREDIT', inplace = True)

In [ ]:
order_df_new =order_df[['account_id','k_symbol']]
order_df_new.drop_duplicates(inplace=True)
order_df_new=order_df_new.reset_index(drop=True)
order_df_new_dummy=pd.get_dummies(order_df_new, columns=["k_symbol"]).groupby(['account_id'], as_index=False).sum()

### Creating features & calculating credit and debit balance from each account id

In [ ]:
f = lambda x: -x

In [ ]:
#columns with amounts that affects balance positively - collection from another banks 
trans_df = trans_df.assign (pos_sum = np.nan)

#columns with amounts that affects balance negatively - remittance to another banks
trans_df = trans_df.assign (neg_sum = np.nan)

#filling columns pos_sum and neg_sum based on operation type. pos_sum stands for inflows
trans_df.pos_sum = trans_df.amount.where (trans_df.type == 'Credit', trans_df.pos_sum)

#neg_sum stands for outflows
trans_df.neg_sum = trans_df.amount.where (trans_df.type == 'Debit', trans_df.neg_sum )

In [ ]:
trans_df['neg_sum']=trans_df['neg_sum'].apply(f)
dummy_trans_df= trans_df[['account_id', 'type', 'fulldate','pos_sum', 'neg_sum']]

In [ ]:
dummy_trans_cr_db_df=dummy_trans_df.groupby(['account_id', 'type']).sum() \
  .groupby(level=0).cumsum().reset_index()
dummy_trans_db_df =dummy_trans_cr_db_df[dummy_trans_cr_db_df.type=='Debit']

In [ ]:
dummy_trans_db_df.head(3)

,account_id,type,pos_sum,neg_sum
1,A00000001,Debit,194320.5,-180854.0
3,A00000002,Debit,1597053.5,-1554425.8
5,A00000003,Debit,173059.0,-121962.8


#### Creating dummies of operation performed by each Account to merge in tans_df 

In [ ]:
df_dummy=trans_df[['account_id','operation']]
df_dummy.drop_duplicates(inplace=True) 
df_dummy=df_dummy.reset_index(drop=True)

Trans_operat_df_dummy=pd.get_dummies(df_dummy, columns=["operation"]).groupby(['account_id'], as_index=False).sum()

In [ ]:
Trans_operat_df_dummy.head()

,account_id,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank
0,A00000001,1,1,0,1,1,1
1,A00000002,1,1,0,1,1,1
2,A00000003,1,0,0,1,1,1
3,A00000004,1,1,0,1,1,1
4,A00000005,1,1,0,1,1,1


### MERGE 1
1. dummy_trans_db_df + Trans_operat_df_dummy

In [ ]:

Trans_merged_df = dummy_trans_db_df.merge (Trans_operat_df_dummy, left_on = 'account_id', right_on = 'account_id', 
                                           how = 'left', validate = 'one_to_one') 
display (dummy_trans_db_df.head (n=3))
display (Trans_merged_df.head (n=3))     
print ("Verifying shape. Before: " + str (dummy_trans_db_df.shape)+ " After: " + str (Trans_merged_df.shape))

,account_id,type,pos_sum,neg_sum
1,A00000001,Debit,194320.5,-180854.0
3,A00000002,Debit,1597053.5,-1554425.8
5,A00000003,Debit,173059.0,-121962.8


,account_id,type,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1
2,A00000003,Debit,173059.0,-121962.8,1,0,0,1,1,1


Verifying shape. Before: (4500, 4) After: (4500, 10)


### MERGE 2
1. Merge1 + accounts_df

In [ ]:

#merging accounts_df and corresponding Trans_merged_df 

trans_accnt_df = Trans_merged_df.merge (accounts_df, left_on = 'account_id', right_on = 'account_id', 
                                           how = 'left', validate = 'one_to_one') 
display (Trans_merged_df.head (n=3))
display (trans_accnt_df.head (n=3))     
print ("Verifying shape. Before: " + str (Trans_merged_df.shape)+ " After: " + str (trans_accnt_df.shape))

,account_id,type,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1
2,A00000003,Debit,173059.0,-121962.8,1,0,0,1,1,1


,account_id,type,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id,frequency,parseddate,year,month,day
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26
2,A00000003,Debit,173059.0,-121962.8,1,0,0,1,1,1,5,Monthly Issuance,2017-07-07,2017,7,7


Verifying shape. Before: (4500, 10) After: (4500, 16)


In [ ]:
#merging accounts_df and corresponding order_df_summery 

trans_accnt_df = trans_accnt_df.merge (order_df_new_dummy, left_on = 'account_id', right_on = 'account_id', 
                                           how = 'left', validate = 'one_to_one') 
display (trans_accnt_df.head (n=3))     
print ("Verifying shape.  After: " + str (trans_accnt_df.shape))

,account_id,type,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0
2,A00000003,Debit,173059.0,-121962.8,1,0,0,1,1,1,5,Monthly Issuance,2017-07-07,2017,7,7,1.0,1.0,1.0,0.0,0.0


Verifying shape.  After: (4500, 21)


### MERGE 3
1. Merge2 + dispos_df

In [ ]:
dispos_dummy_df=pd.get_dummies(dispos_df, columns=["type"]).groupby(['client_id','account_id','disp_id'], as_index=False).sum()

In [ ]:
#merging cards_df and corresponding dispos_df so to find client_id and then from client_id to find corresponding region data

trans_accnt_dispos_df = trans_accnt_df.merge (dispos_dummy_df, left_on = 'account_id', right_on = 'account_id', 
                                           how = 'left', validate = 'one_to_many') 
display (trans_accnt_df.head (n=3))
display (trans_accnt_dispos_df.head (n=3))     
print ("Verifying shape. Before: " + str (trans_accnt_df.shape)+ " After: " + str (trans_accnt_dispos_df.shape))

,account_id,type,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0
2,A00000003,Debit,173059.0,-121962.8,1,0,0,1,1,1,5,Monthly Issuance,2017-07-07,2017,7,7,1.0,1.0,1.0,0.0,0.0


,account_id,type,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment,client_id,disp_id,type_Owner,type_User
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0,C00000001,D00000001,1,0
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,C00000002,D00000002,1,0
2,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,C00000003,D00000003,0,1


Verifying shape. Before: (4500, 21) After: (5369, 25)


### Merge 4
merge 3+Cards_df

In [ ]:
cards_df =cards_df[['disp_id', 'type']]

In [ ]:
trans_accnt_dispos_cards_df = trans_accnt_dispos_df.merge (cards_df, left_on = 'disp_id', right_on = 'disp_id', 
                                           how = 'left', validate = 'many_to_one') 
display (trans_accnt_dispos_df.head (n=3))
display (trans_accnt_dispos_cards_df.head (n=3))     
print ("Verifying shape. Before: " + str (trans_accnt_dispos_df.shape)+ " After: " + str (trans_accnt_dispos_cards_df.shape))

,account_id,type,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment,client_id,disp_id,type_Owner,type_User
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0,C00000001,D00000001,1,0
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,C00000002,D00000002,1,0
2,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,C00000003,D00000003,0,1


,account_id,type_x,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment,client_id,disp_id,type_Owner,type_User,type_y
0,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0,C00000001,D00000001,1,0,NaN
1,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,C00000002,D00000002,1,0,NaN
2,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,C00000003,D00000003,0,1,NaN


Verifying shape. Before: (5369, 25) After: (5369, 26)


### Merge 5
merge 4+Clients_df

In [ ]:
clients_dist_df = clients_df
clients_dist2_df = clients_dist_df.merge (district_df, left_on = 'district_id', right_on = 'district_id', 
                                           how = 'left', validate = 'many_to_one') 
display (clients_df.head (n=3))
display (clients_dist2_df.head (n=3))

,client_id,sex,fulldate,day,month,year,age,social,first,middle,last,phone,email,address_1,address_2,city,state,zipcode,district_id
0,C00000001,Female,1990-12-13,13,12,1990,29,926-93-2157,Emma,Avaya,Smith,367-171-6840,emma.smith@gmail.com,387 Wellington Ave.,Unit 1,Albuquerque,NM,47246,18
1,C00000002,Male,1965-02-04,4,2,1965,54,806-94-5725,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040,1
2,C00000003,Female,1960-10-09,9,10,1960,59,614-70-9100,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162,1


,client_id,sex,fulldate,day,month,year,age,social,first,middle,last,phone,email,address_1,address_2,city_x,state,zipcode,district_id,city_y,state_name,state_abbrev,region,division
0,C00000001,Female,1990-12-13,13,12,1990,29,926-93-2157,Emma,Avaya,Smith,367-171-6840,emma.smith@gmail.com,387 Wellington Ave.,Unit 1,Albuquerque,NM,47246,18,Albuquerque,New Mexico,NM,West,Mountain
1,C00000002,Male,1965-02-04,4,2,1965,54,806-94-5725,Noah,Everest,Thompson,212-423-7734,noah.thompson@gmail.com,75 W. Berkshire St.,NaN,New York City,NY,10040,1,New York City,New York,NY,Northeast,Middle Atlantic
2,C00000003,Female,1960-10-09,9,10,1960,59,614-70-9100,Olivia,Brooklynne,Johnson,212-425-6932,olivia.johnson@outlook.com,36 Second St.,NaN,New York City,NY,10162,1,New York City,New York,NY,Northeast,Middle Atlantic


In [ ]:
clients_df_dummy =clients_dist2_df[['client_id', 'sex','age','city_x', 'state', 'zipcode', 'district_id','region','division']]

In [ ]:
final_df = clients_df_dummy.merge (trans_accnt_dispos_cards_df, left_on = 'client_id', right_on = 'client_id', 
                                           how = 'left', validate = 'one_to_one') 
display (clients_df_dummy.head (n=3))
display (final_df.head (n=3))     
print ("Verifying shape. Before: " + str (clients_df_dummy.shape)+ " After: " + str (final_df.shape))

,client_id,sex,age,city_x,state,zipcode,district_id,region,division
0,C00000001,Female,29,Albuquerque,NM,47246,18,West,Mountain
1,C00000002,Male,54,New York City,NY,10040,1,Northeast,Middle Atlantic
2,C00000003,Female,59,New York City,NY,10162,1,Northeast,Middle Atlantic


,client_id,sex,age,city_x,state,zipcode,district_id_x,region,division,account_id,type_x,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id_y,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment,disp_id,type_Owner,type_User,type_y
0,C00000001,Female,29,Albuquerque,NM,47246,18,West,Mountain,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0,D00000001,1,0,NaN
1,C00000002,Male,54,New York City,NY,10040,1,Northeast,Middle Atlantic,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,D00000002,1,0,NaN
2,C00000003,Female,59,New York City,NY,10162,1,Northeast,Middle Atlantic,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,D00000003,0,1,NaN


Verifying shape. Before: (5369, 9) After: (5369, 34)


In [ ]:
crm_event_df = pd.read_excel('CRM Events.xlsx')

In [ ]:
crm_event_df_new =crm_event_df[['Product', 'Sub-product','Client_ID']]
mask = crm_event_df_new['Sub-product'].isnull()
crm_event_df_new.loc[mask, 'Sub-product'] = crm_event_df_new.loc[mask, 'Product'].map({'Credit card':'Insurence', 'Bank account or service':'Demat'})

crm_event_df_new.drop_duplicates(inplace=True)
crm_event_df_new=crm_event_df_new.reset_index(drop=True)

In [ ]:
crm_event_df_new.columns

Index(['Product', 'Sub-product', 'Client_ID'], dtype='object')

In [ ]:
d1=crm_event_df_new[['Product','Client_ID']]
d2=crm_event_df_new[['Sub-product','Client_ID']]

d2.rename(columns={'Sub-product':'Product'},inplace=True)
          
frame =[d1,d2]
product_result=pd.concat(frame,ignore_index=True)

In [ ]:
product_result.drop_duplicates(inplace=True)
product_result=product_result.reset_index(drop=True)

In [ ]:
product_result_dummy =pd.get_dummies(product_result, columns=['Product']).groupby(['Client_ID'], as_index=False).sum()

In [ ]:
final_df_products = final_df.merge (product_result_dummy, left_on = 'client_id', right_on = 'Client_ID', 
                                           how = 'left', validate = 'one_to_one') 
display (final_df.head (n=3))
display (final_df_products.head (n=3))     
print ("Verifying shape. Before: " + str (final_df.shape)+ " After: " + str (final_df_products.shape))

,client_id,sex,age,city_x,state,zipcode,district_id_x,region,division,account_id,type_x,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id_y,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment,disp_id,type_Owner,type_User,type_y
0,C00000001,Female,29,Albuquerque,NM,47246,18,West,Mountain,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0,D00000001,1,0,NaN
1,C00000002,Male,54,New York City,NY,10040,1,Northeast,Middle Atlantic,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,D00000002,1,0,NaN
2,C00000003,Female,59,New York City,NY,10162,1,Northeast,Middle Atlantic,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,D00000003,0,1,NaN


,client_id,sex,age,city_x,state,zipcode,district_id_x,region,division,account_id,type_x,pos_sum,neg_sum,operation_Cash Withdrawal,operation_Collection from Another Bank,operation_Credit Card Withdrawal,operation_Credit in Cash,operation_INTEREST_CREDIT,operation_Remittance to Another Bank,district_id_y,frequency,parseddate,year,month,day,k_symbol_,k_symbol_Household Payment,k_symbol_Insurance Payment,k_symbol_Leasing Payment,k_symbol_Loan Payment,disp_id,type_Owner,type_User,type_y,Client_ID,Product_(CD) Certificate of deposit,Product_Bank account or service,Product_Cashing a check without an account,Product_Checking account,Product_Credit card,Product_Insurence,Product_Other bank product/service,Product_Savings account
0,C00000001,Female,29,Albuquerque,NM,47246,18,West,Mountain,A00000001,Debit,194320.5,-180854.0,1,1,0,1,1,1,18,Monthly Issuance,2015-03-24,2015,3,24,0.0,1.0,0.0,0.0,0.0,D00000001,1,0,NaN,C00000001,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
1,C00000002,Male,54,New York City,NY,10040,1,Northeast,Middle Atlantic,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,D00000002,1,0,NaN,C00000002,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,C00000003,Female,59,New York City,NY,10162,1,Northeast,Middle Atlantic,A00000002,Debit,1597053.5,-1554425.8,1,1,0,1,1,1,1,Monthly Issuance,2013-02-26,2013,2,26,0.0,1.0,0.0,0.0,1.0,D00000003,0,1,NaN,C00000003,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0


Verifying shape. Before: (5369, 34) After: (5369, 43)


### Imputations

In [ ]:

mask = final_df_products['type_y'].isnull()
final_df_products.loc[mask, 'type_y'] = final_df_products.loc[mask, 'operation_Credit Card Withdrawal'].map({1:'VISA Infinite', 0:'No CreditCard'})

In [ ]:
final_df_products =pd.get_dummies(final_df_products, columns=['type_y'])

In [ ]:
final_df_products['loanexist']=np.where(final_df_products['account_id'].isin(loan_df['account_id']),1,0)

In [ ]:
final_df_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5369 entries, 0 to 5368
Data columns (total 47 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   client_id                                   5369 non-null   object 
 1   sex                                         5369 non-null   object 
 2   age                                         5369 non-null   int64  
 3   city_x                                      5369 non-null   object 
 4   state                                       5369 non-null   object 
 5   zipcode                                     5369 non-null   int64  
 6   district_id_x                               5369 non-null   int64  
 7   region                                      5369 non-null   object 
 8   division                                    5369 non-null   object 
 9   account_id                                  5369 non-null   object 
 10  type_x      

In [ ]:
final_df_products.drop(['account_id','district_id_y','disp_id','type_x','parseddate','k_symbol_ ','Client_ID'],axis=1,inplace=True)


In [ ]:
cols =['Product_(CD) Certificate of deposit',
       'Product_Bank account or service',
       'Product_Cashing a check without an account',
       'Product_Checking account', 'Product_Credit card', 'Product_Insurence',
       'Product_Other bank product/service', 'Product_Savings account','k_symbol_Household Payment',
       'k_symbol_Insurance Payment', 'k_symbol_Leasing Payment',
       'k_symbol_Loan Payment']

In [ ]:
final_df_products[cols] = final_df_products[cols].fillna(0.0).astype(int)

In [ ]:
final_df_products.info()

In [ ]:
pip install turicreate 

In [ ]:
import time
import turicreate as tc
from sklearn.model_selection import train_test_split


In [ ]:
final_df_dummy =final_df_products[['client_id','Product_(CD) Certificate of deposit',
       'Product_Bank account or service',
       'Product_Cashing a check without an account',
       'Product_Checking account', 'Product_Credit card', 'Product_Insurence',
       'Product_Other bank product/service', 'Product_Savings account',
       'type_y_No CreditCard', 'type_y_VISA Infinite', 'type_y_VISA Signature',
       'type_y_VISA Standard', 'loanexist']]

In [ ]:
final_df_dummy=final_df_dummy.set_index('client_id')

In [ ]:
final_df_dummy

,Product_(CD) Certificate of deposit,Product_Bank account or service,Product_Cashing a check without an account,Product_Checking account,Product_Credit card,Product_Insurence,Product_Other bank product/service,Product_Savings account,type_y_No CreditCard,type_y_VISA Infinite,type_y_VISA Signature,type_y_VISA Standard,loanexist
client_id,,,,,,,,,,,,,
C00000001,0,1,0,1,1,1,1,0,1,0,0,0,0
C00000002,0,0,0,0,1,1,0,0,1,0,0,0,1
C00000003,0,1,0,1,1,1,1,0,1,0,0,0,1
C00000004,0,1,0,1,1,1,1,0,1,0,0,0,0
C00000005,0,1,0,1,1,1,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
C00013955,0,1,0,1,1,1,0,0,1,0,0,0,1
C00013956,0,1,0,1,0,0,1,0,1,0,0,0,1
C00013968,0,1,0,1,0,0,0,1,0,0,1,0,1


In [ ]:
final_df_dummy_labels = final_df_dummy.stack()
final_df_dummy_labels

client_id                                            
C00000001  Product_(CD) Certificate of deposit           0
           Product_Bank account or service               1
           Product_Cashing a check without an account    0
           Product_Checking account                      1
           Product_Credit card                           1
                                                        ..
C00013998  type_y_No CreditCard                          1
           type_y_VISA Infinite                          0
           type_y_VISA Signature                         0
           type_y_VISA Standard                          0
           loanexist                                     0
Length: 69797, dtype: int64

In [ ]:
final_df_dummy_labels = final_df_dummy_labels.reset_index()
final_df_dummy_labels

,client_id,level_1,0
0,C00000001,Product_(CD) Certificate of deposit,0
1,C00000001,Product_Bank account or service,1
2,C00000001,Product_Cashing a check without an account,0
3,C00000001,Product_Checking account,1
4,C00000001,Product_Credit card,1
...,...,...,...
69792,C00013998,type_y_No CreditCard,1
69793,C00013998,type_y_VISA Infinite,0
69794,C00013998,type_y_VISA Signature,0
69795,C00013998,type_y_VISA Standard,0


In [ ]:
final_df_dummy_labels.columns = ["client_id", "product", "is_owned"]

In [ ]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
train_data, test_data = split_data(final_df_dummy_labels)

In [ ]:
user_id = 'client_id'
item_id = 'product'
users_to_recommend = list(final_df_products[user_id])
n_rec = 5 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [ ]:
user_info = tc.SFrame({'client_id': list(final_df_products['client_id']),
                                'age_category': list(final_df_products['age']),
                                'gender': list(final_df_products['sex'])  ,
                               'city_x': list(final_df_products['city_x'])    ,
                                'state':  list(final_df_products['state'])         ,
                                'zipcode': list(final_df_products['zipcode'])         ,
                                'district_id_x': list(final_df_products['district_id_x'])        ,
                               'region':  list(final_df_products['region'])       ,
                                 'division':  list(final_df_products['division'])          ,
                                 'pos_sum':  list(final_df_products['pos_sum'])           ,
                                'neg_sum':  list(final_df_products['neg_sum']),
                                'frequency':  list(final_df_products['frequency'])    
                       })

In [ ]:
def model(train_data, name, user_id, item_id, target, user_feature,users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target,
                                                    user_data=user_info
                                                 )
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    user_data =user_info,
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    user_data =user_info,
                                                    similarity_type='pearson')
    elif name == 'factorization':
        model = tc.factorization_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    user_data =user_info          
                                                    )
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
name = 'factorization'
target = 'is_owned'
factor = model(train_data, name, user_id, item_id, target,user_feature,users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 55837 observations with 5369 users and 13 items.

Data prepared in: 0.123853s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

recommendations finished on 2000/5369 queries. users per second: 349223

recommendations finished on 3000/5369 queries. users per second: 286205

recommendations finished on 4000/5369 queries. users per second: 277701

recommendations finished on 5000/5369 queries. users per second: 271312

+-----------+-------------------------------+----------------------+------+
| client_id |            product            |        score         | rank |
+-----------+-------------------------------+----------------------+------+
| C00000001 |      Product_Credit card      |  0.8853545861596674  |  1   |
| C00000002 |       Product_Insurence       |  0.6516356681330471  |  1   |
| C00000002 |      type_y_No CreditCard     | 0.23177384504548892  |  2   |
| C00000002 |    Product_Savings account    | -0.04496593877241317 |  3   |
| C00000003 |       Product_Insurence       |  0.9768891688958439  |  1   |
| C00000003 | Product_Other bank product... |  0.4418627732024223  |  2   |
| C00000003 |     type_y_VISA Signature     | 0.15782103506481948  |  3   |
| C00000003 | Product_Cashing a check wi... | 0.10398275596654716  |  4   |
| C00000004 |           loanexist           | 0.10710826342446012  |  1   |
| C00000004 |      type_y_VISA Infinite     |  0.0876065959547202  |  2   |
| C00000005 

In [ ]:
name = 'popularity'
target = 'is_owned'
popularity = model(train_data, name, user_id, item_id, target,user_feature,users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 55837 observations with 5369 users and 13 items.

Data prepared in: 0.117912s

55837 observations to process; with 13 unique items.

recommendations finished on 1000/5369 queries. users per second: 392619

recommendations finished on 2000/5369 queries. users per second: 441404

recommendations finished on 3000/5369 queries. users per second: 320171

recommendations finished on 4000/5369 queries. users per second: 308737

recommendations finished on 5000/5369 queries. users per second: 330710

+-----------+-------------------------------+----------------------+------+
| client_id |            product            |        score         | rank |
+-----------+-------------------------------+----------------------+------+
| C00000001 |      Product_Credit card      |  0.8198427382053654  |  1   |
| C00000002 |       Product_Insurence       |  0.8243685687558466  |  1   |
| C00000002 |      type_y_No CreditCard     |  0.8096683792620271  |  2   |
| C00000002 |    Product_Savings account    | 0.13716295427901523  |  3   |
| C00000003 |       Product_Insurence       |  0.8243685687558466  |  1   |
| C00000003 | Product_Other bank product... | 0.40060875673144464  |  2   |
| C00000003 |     type_y_VISA Signature     | 0.12594980428275385  |  3   |
| C00000003 | Product_Cashing a check wi... | 0.021311475409836064 |  4   |
| C00000004 |           loanexist           | 0.15251099282573477  |  1   |
| C00000004 |      type_y_VISA Infinite     | 0.04053420805998126  |  2   |
| C00000005 

In [ ]:
name = 'cosine'
target = 'is_owned'
cos = model(train_data, name, user_id, item_id, target, user_feature,users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 55837 observations with 5369 users and 13 items.

Data prepared in: 0.125754s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 2000/5369 queries. users per second: 221166

recommendations finished on 3000/5369 queries. users per second: 239617

recommendations finished on 4000/5369 queries. users per second: 248927

recommendations finished on 5000/5369 queries. users per second: 250225

+-----------+-------------------------------+----------------------+------+
| client_id |            product            |        score         | rank |
+-----------+-------------------------------+----------------------+------+
| C00000001 |      Product_Credit card      | 0.27419742445151013  |  1   |
| C00000002 |       Product_Insurence       | 0.10854117870330811  |  1   |
| C00000002 |      type_y_No CreditCard     | 0.09211570620536805  |  2   |
| C00000002 |    Product_Savings account    | 0.03964138627052307  |  3   |
| C00000003 |       Product_Insurence       | 0.34529300530751544  |  1   |
| C00000003 | Product_Other bank product... | 0.23597502046161228  |  2   |
| C00000003 |     type_y_VISA Signature     | 0.10071263048383924  |  3   |
| C00000003 | Product_Cashing a check wi... | 0.055515673425462514 |  4   |
| C00000004 |           loanexist           | 0.14881006696007468  |  1   |
| C00000004 |      type_y_VISA Infinite     | 0.06233603845943104  |  2   |
| C00000005 

In [ ]:
models_w_counts = [popularity, cos, pear,factor]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts','factorization on Purchase Counts']

In [ ]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/5079 queries. users per second: 218484

recommendations finished on 2000/5079 queries. users per second: 242454

recommendations finished on 3000/5079 queries. users per second: 231374

recommendations finished on 4000/5079 queries. users per second: 213288

...

recommendations finished on 2000/5079 queries. users per second: 232829

recommendations finished on 3000/5079 queries. users per second: 210482

recommendations finished on 4000/5079 queries. users per second: 219443

recommendations finished on 5000/5079 queries. users per second: 236474


Precision and recall summary statistics by cutoff
+--------+----------------+---------------------+
| cutoff | mean_precision |     mean_recall     |
+--------+----------------+---------------------+
|   1    |      1.0       | 0.48056586879681895 |
|   2    |      1.0       |  0.7654239210943273 |
|   3    |      1.0       |  0.914428458920483  |
|   4    |      1.0       |  0.973257765401891  |
|   5    |      1.0       |  0.9933983536316677 |
|   6    |      1.0       |  0.9988116333361458 |
|   7    |      1.0       |  0.9998277219925182 |
|   8    |      1.0       |         1.0         |
|   9    |      1.0       |         1.0         |
|   10   |      1.0       |         1.0         |
+--------+----------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.33022448280076094

Per User RMSE (best)
+-----------+----------------------+-------+
| client_id |         rmse         | count |
+-----------+----------------------+-------+
| C00007863 | 0.000375619985270248

In [ ]:
final_model = tc.popularity_recommender.create(tc.SFrame(test_data), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='is_owned',user_data=user_info)


Preparing data set.

Data has 13960 observations with 5369 users and 13 items.

Data prepared in: 0.078468s

13960 observations to process; with 13 unique items.

In [ ]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

recommendations finished on 1000/5369 queries. users per second: 347464

recommendations finished on 2000/5369 queries. users per second: 344234

recommendations finished on 3000/5369 queries. users per second: 285687

recommendations finished on 4000/5369 queries. users per second: 318142

recommendations finished on 5000/5369 queries. users per second: 311935

+-----------+-------------------------------+---------------------+------+
| client_id |            product            |        score        | rank |
+-----------+-------------------------------+---------------------+------+
| C00000001 | Product_Bank account or se... |  0.9235912129894938 |  1   |
| C00000001 |    Product_Checking account   |  0.8404558404558404 |  2   |
| C00000001 |      type_y_No CreditCard     |  0.8077276908923643 |  3   |
| C00000001 |       Product_Insurence       |  0.8060384263494969 |  4   |
| C00000001 | Product_Other bank product... |  0.4007285974499089 |  5   |
| C00000002 | Product_Bank account or se... |  0.9235912129894938 |  1   |
| C00000002 |    Product_Checking account   |  0.8404558404558404 |  2   |
| C00000002 |      Product_Credit card      |  0.8239234449760765 |  3   |
| C00000002 | Product_Other bank product... |  0.4007285974499089 |  4   |
| C00000002 |           loanexist           | 0.16030534351145037 |  5   |
| C00000003 | Product_Ban

In [ ]:
recom

client_id,product,score,rank
C00000001,Product_Bank account orservice ...,0.9235912129894938,1
C00000001,Product_Checking account,0.8404558404558404,2
C00000001,type_y_No CreditCard,0.8077276908923643,3
C00000001,Product_Insurence,0.8060384263494969,4
C00000001,Product_Other bankproduct/service ...,0.4007285974499089,5
C00000002,Product_Bank account orservice ...,0.9235912129894938,1
C00000002,Product_Checking account,0.8404558404558404,2
C00000002,Product_Credit card,0.8239234449760765,3
C00000002,Product_Other bankproduct/service ...,0.4007285974499089,4
C00000002,loanexist,0.16030534351145037,5


In [ ]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(26845, 4)


,client_id,product,score,rank
0,C00000001,Product_Bank account or service,0.923591,1
1,C00000001,Product_Checking account,0.840456,2
2,C00000001,type_y_No CreditCard,0.807728,3
3,C00000001,Product_Insurence,0.806038,4
4,C00000001,Product_Other bank product/service,0.400729,5


In [ ]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['client_id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('client_id').set_index('client_id')
    if print_csv:
        df_output.to_csv('option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [ ]:
df_output = create_output(final_model, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/5369 queries. users per second: 304414

recommendations finished on 2000/5369 queries. users per second: 316807

recommendations finished on 3000/5369 queries. users per second: 260892

recommendations finished on 4000/5369 queries. users per second: 252270

recommendations finished on 5000/5369 queries. users per second: 262715

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(5369, 1)


,recommendedProducts
client_id,
C00000001,Product_Bank account or service|Product_Checki...
C00000002,Product_Bank account or service|Product_Checki...
C00000003,Product_Bank account or service|Product_Checki...
C00000004,Product_Bank account or service|Product_Checki...
C00000005,Product_Bank account or service|Product_Checki...


In [ ]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [ ]:
customer_recomendation('C00000008')

recommendedProducts    Product_Bank account or service|Product_Checki...
Name: C00000008, dtype: object

In [2]:
pip install pyre-check

     |████████████████████████████████| 24.6MB 169kB/s 
     |████████████████████████████████| 501kB 36.8MB/s 
     |████████████████████████████████| 276kB 42.4MB/s 
  Created wheel for pywatchman: filename=pywatchman-1.4.1-cp36-cp36m-linux_x86_64.whl size=58063 sha256=9e5c516dea33c21f2ac62f473c9714d21858a8463031882eb0f3dab66d06cae5
  Stored in directory: /root/.cache/pip/wheels/7f/ee/43/9359d63838983e133cf93c3c600ca7d5fc3c75718a2e86f011
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=805e43ffc65b8652437b4ebf4cbf90996ecf26fc2e5cb01a3284a8f43e4fbcb2
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pywatchman pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
pip install click click-log ipython==7.6.1 munch pygments SQLAlchemy ujson~=1.35 xxhash~=1.3.0 prompt-toolkit~=2.0.9 flask flask_cors flask_graphql graphene graphene_sqlalchemy


     |████████████████████████████████| 778kB 3.2MB/s 
     |████████████████████████████████| 194kB 17.1MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
     |████████████████████████████████| 348kB 19.5MB/s 
     |████████████████████████████████| 112kB 20.7MB/s 
     |████████████████████████████████| 256kB 18.6MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 184kB 20.8MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68024 sha256=54ff8c03361cca6e1e9c159ab5c9c987b2d875c95622e51260f18bf72e72ca26
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
  Created wheel for flask-graphql: filename=Flask_GraphQL-2.0.1-cp36-none-any.whl size=6366 sha256=a000c5db105e18543d3348282bea8f8e0bd1041215609be321ee42d70155d809
  Stored in directory: /root/.cache/pip/wheels/59/ed/06/1b237b69f872b7c24d1c0c9996cc76de0af003d50e97defd2a
  Created wheel for g

In [6]:
pip install pyre-check


Object `pyre` not found.
